# Import thư viện

In [102]:
import pandas as pd
import numpy as np

# Đọc file để xử lý ( fdi, cpi, gdp, retail, unemployment)

In [103]:
df_cpi = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\cpi_index_VN.csv')
df_fdi = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\fdi_inflow_VN.csv')
df_gdp = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\gdp_growth_VN.csv')
df_retail = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\Retail_processed.csv.csv')
df_unemployment = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\Unemployment_processed.csv')

# Định dạng lại cột date và chuẩn bị dữ liệu để Resample


In [ ]:
def prepare_data(df, date_col='date', value_col=None, start_year=2010, end_year=2024):
    """
    🧩 Mục đích:
    - Chuyển cột ngày ('date') sang định dạng datetime
    - Thiết lập nó làm index thời gian (Time Series Index)
    - Lọc dữ liệu trong khoảng năm chỉ định
    - Đảm bảo chỉ giữ lại 1 cột giá trị chính (value)
    
    Thường dùng để chuẩn hóa dữ liệu kinh tế từ World Bank, Yahoo Finance, CafeF, v.v.
    """

    # 1️⃣ Chuyển cột 'date' về dạng datetime để dễ xử lý thời gian
    df[date_col] = pd.to_datetime(df[date_col])

    # 2️⃣ Thiết lập cột 'date' làm index của DataFrame (Time Index)
    df = df.set_index(date_col)

    # 3️⃣ Nếu chưa chỉ định cột giá trị → chọn cột đầu tiên trong DataFrame
    if value_col is None:
        value_col = df.columns[0]

    # 4️⃣ Chỉ giữ lại một cột dữ liệu chính, đổi tên thành 'annual_value' để đồng nhất
    df = df[[value_col]].rename(columns={value_col: 'annual_value'})

    # 5️⃣ Sắp xếp index theo thứ tự thời gian tăng dần (đảm bảo thứ tự đúng)
    df = df.sort_index()

    # 6️⃣ Lọc dữ liệu theo khoảng năm yêu cầu
    df = df[(df.index.year >= start_year) & (df.index.year <= end_year)]

    # 7️⃣ Trả về DataFrame đã chuẩn hóa
    return df


# -----------------------------
# 🧾 CHUẨN BỊ DỮ LIỆU CÁC BIẾN KINH TẾ
# -----------------------------

# Mỗi DataFrame (df_fdi, df_cpi, df_gdp, ...) đã được crawl từ các nguồn khác nhau
# Ta chuẩn hóa toàn bộ bằng hàm prepare_data()
data_frames = {
    'fdi_inflow': prepare_data(df_fdi, value_col='fdi_inflow'),                  # Dòng vốn đầu tư nước ngoài
    'cpi_index': prepare_data(df_cpi, value_col='cpi_index'),                    # Chỉ số giá tiêu dùng (lạm phát)
    'gdp_growth': prepare_data(df_gdp, value_col='gdp_percent'),                 # Tăng trưởng GDP
    'retail_value': prepare_data(df_retail, value_col='retail_value'),           # Giá trị bán lẻ
    'unemployment_value': prepare_data(df_unemployment, value_col='unemployment_value')  # Tỷ lệ thất nghiệp
}


# -----------------------------
# ⚙️ ĐỊNH NGHĨA THAM SỐ TỔNG HỢP DỮ LIỆU (DISAGG_PARAMS)
# -----------------------------

# Vì các dữ liệu này có thể khác tần suất (VD: quý / năm),
# nên ta định nghĩa quy tắc tổng hợp (aggregation rule) để quy đổi về cùng tần suất.
DISAGG_PARAMS = {
    # Nhóm 1️⃣: Dữ liệu dạng *dòng chảy (Flow)* → ví dụ FDI, xuất nhập khẩu, doanh thu
    # Tổng 4 quý trong năm = giá trị năm
    'fdi_inflow': {'agg_func': 'sum'},

    # Nhóm 2️⃣: Dữ liệu dạng *tỷ lệ hoặc chỉ số (Rate/Index)* → ví dụ CPI, GDP Growth
    # Lấy trung bình của 4 quý = giá trị năm
    'cpi_index': {'agg_func': 'mean'},
    'gdp_growth': {'agg_func': 'mean'},
    'retail_value': {'agg_func': 'mean'},
    'unemployment_value': {'agg_func': 'mean'}
}


# Hàm phân tách dữ liệu từ năm thành quý

In [ ]:
def benchmarked_linear_disaggregation(df_annual, agg_func):
    """
    Mục tiêu:
    - Phân rã dữ liệu NĂM → QUÝ bằng nội suy tuyến tính.
    - Sau đó 'benchmark' (chuẩn hóa) để đảm bảo:
        * Nếu agg_func='sum'  : TỔNG 4 quý = giá trị năm
        * Nếu agg_func='mean' : TRUNG BÌNH 4 quý = giá trị năm
    Yêu cầu:
    - df_annual có index thời gian (năm) và cột 'annual_value'.
    - agg_func ∈ {'sum', 'mean'}.
    Trả về:
    - pandas.Series theo tần suất QUÝ (DatetimeIndex), tên 'quarterly_value'.
    """
    value_col = 'annual_value'
    
    # Bảo đảm index được sắp xếp tăng dần theo thời gian
    df_annual = df_annual.sort_index()
    
    # (Tuỳ chọn) kiểm tra tham số hợp lệ
    if agg_func not in ('sum', 'mean'):
        raise ValueError("agg_func phải là 'sum' hoặc 'mean'.")

    # ===============================
    # 1) UPSAMPLING & NỘI SUY CƠ SỞ
    # ===============================
    # Đưa index về mốc ĐẦU NĂM (YYYY-01-01) để resample chính xác theo quý
    df_annual_start = df_annual.copy()
    df_annual_start.index = df_annual_start.index.to_period('Y').to_timestamp(how='start')
    # Đặt tên cho index để reset_index sau này có cột 'date' rõ ràng
    df_annual_start.index.name = 'date'

    # Resample sang QUÝ (Quarter Start), tạo khung thời gian quý rỗng
    df_quarterly_resampled = df_annual_start.resample('QS').asfreq()

    # Nội suy tuyến tính các điểm quý giữa hai mốc năm
    df_quarterly_interpolated = df_quarterly_resampled.interpolate(method='linear')
    df_quarterly_interpolated.columns = ['interpolated_value']  # đặt tên cột rõ ràng

    # ===============================
    # 2) CHUẨN BỊ DỮ LIỆU BENCHMARK
    # ===============================
    # Tạo 'annual_period' (kiểu Period[Y]) để gom nhóm theo năm
    df_annual_temp = df_annual.copy()
    df_annual_temp['annual_period'] = df_annual_temp.index.to_period('Y')
    df_quarterly_interpolated['annual_period'] = df_quarterly_interpolated.index.to_period('Y')

    # Bản đồ giá trị năm thật (ground-truth) theo annual_period
    annual_map = df_annual_temp.set_index('annual_period')[value_col]

    # Tổng/Trung bình giá trị quý nội suy theo từng năm (tuỳ agg_func)
    agg_quarterly = (
        df_quarterly_interpolated
        .groupby('annual_period')['interpolated_value']
        .agg(agg_func)
    )

    # Gom vào dataframe để tính hệ số điều chỉnh
    df_annual_adjustment = pd.DataFrame({
        'annual_value'    : annual_map,        # giá trị năm mục tiêu
        'agg_interpolated': agg_quarterly      # tổng/trung bình của chuỗi quý nội suy
    }).dropna()  # chỉ giữ năm có đủ cả hai vế

    # ===============================
    # 3) TÍNH HỆ SỐ ĐIỀU CHỈNH (ADJUSTMENT)
    # ===============================
    if agg_func == 'sum':
        # Với Sum: dùng hệ số NHÂN để đảm bảo tổng Q1..Q4 == annual_value
        df_annual_adjustment['adjustment'] = (
            df_annual_adjustment['annual_value'] /
            df_annual_adjustment['agg_interpolated'].replace(0, np.nan)  # tránh chia 0
        )
    else:  # 'mean'
        # Với Mean: dùng độ LỆCH CỘNG để đảm bảo mean(Q1..Q4) == annual_value
        df_annual_adjustment['adjustment'] = (
            df_annual_adjustment['annual_value'] -
            df_annual_adjustment['agg_interpolated']
        )

    # ===============================
    # 4) GHÉP HỆ SỐ VÀ ÁP DỤNG ĐIỀU CHỈNH CHO TỪNG QUÝ
    # ===============================
    # reset_index để có cột 'date' (từ index DatetimeIndex)
    df_quarterly_result = df_quarterly_interpolated.reset_index().copy()
    # annual_period đang là Period[Y], chuyển về chuỗi để merge an toàn
    df_quarterly_result['annual_period'] = df_quarterly_result['annual_period'].astype(str)

    adj_reset = df_annual_adjustment['adjustment'].reset_index()
    adj_reset['annual_period'] = adj_reset['annual_period'].astype(str)

    # Merge adjustment theo từng năm
    df_quarterly_result = pd.merge(
        df_quarterly_result,
        adj_reset[['annual_period', 'adjustment']],
        on='annual_period',
        how='left'
    )

    # Tạo giá trị quý sau điều chỉnh:
    if agg_func == 'sum':
        # Phương án nhân (proportional): mỗi quý * ratio
        df_quarterly_result['quarterly_value'] = (
            df_quarterly_result['interpolated_value'] * df_quarterly_result['adjustment']
        )
    else:
        # Phương án cộng (additive): mỗi quý + delta
        df_quarterly_result['quarterly_value'] = (
            df_quarterly_result['interpolated_value'] + df_quarterly_result['adjustment']
        )

    # ===============================
    # 5) HOÀN THIỆN SERIES KẾT QUẢ & LỌC PHẠM VI NĂM
    # ===============================
    # Đặt 'date' làm index, lấy cột kết quả
    result_series = (
        df_quarterly_result
        .set_index('date')['quarterly_value']
        .dropna()
    )

    # Lọc phạm vi năm trong [min_year, max_year] của chuỗi năm gốc
    first_year = df_annual.index.min().year
    last_year  = df_annual.index.max().year
    result_series = result_series[
        (result_series.index.year >= first_year) &
        (result_series.index.year <= last_year)
    ]

    # Thông tin thống kê nhanh
    print(f"  Số quý sau lọc: {len(result_series)}")

    return result_series


In [106]:

# --- ÁP DỤNG VÀ IN KẾT QUẢ ---

results = {}
for name, params in DISAGG_PARAMS.items():
    print(f"Đang xử lý: {name}")
    results[f'{name}_quarterly'] = benchmarked_linear_disaggregation(
        data_frames[name].copy(), 
        agg_func=params['agg_func']
    )

# Kết hợp tất cả dữ liệu
output_df = pd.concat(results.values(), axis=1, join='outer')
output_df.columns = results.keys()


Đang xử lý: fdi_inflow
  Số quý sau lọc: 57
Đang xử lý: cpi_index
  Số quý sau lọc: 57
Đang xử lý: gdp_growth
  Số quý sau lọc: 57
Đang xử lý: retail_value
  Số quý sau lọc: 49
Đang xử lý: unemployment_value
  Số quý sau lọc: 57


# Lưu file dữ liệu quý của 
- fdi  : Vốn Đầu tư Trực tiếp Nước ngoài
- cpi  : Chỉ số Giá tiêu dùng
- gdp  : Tốc độ Tăng trưởng Tổng sản phẩm Quốc nội
- retail : Tổng mức Bán lẻ Hàng hóa và Doanh thu Dịch vụ Tiêu dùng
- unemployment : Tỷ lệ Thất nghiệp

In [107]:
# Lưu kết quả
output_df.to_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\disaggregated_quarterly_data_fixed.csv')


# Đọc file (oil, usdvnd, vnindex)

In [108]:

# 1. Tải các tệp dữ liệu hàng ngày (Daily/Sparse)
df_oil = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\OIL_processed.csv.csv')
df_usdvnd = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\USDVND_processed.csv')
df_vnindex = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\VNINDEX_processed.csv')





# Chuẩn hóa cột date và chuẩn bị dữ liệu để resample

In [109]:

# --- 2. Hàm Chuẩn bị và Làm sạch Dữ liệu Hàng ngày ---
def prepare_daily_data(df, date_col='date'):
    """
    Chuyển đổi cột ngày tháng, thiết lập Index, và xử lý trùng lặp ngày tháng
    (Giữ lại quan sát cuối cùng cho mỗi ngày).
    """
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    
    # Xử lý các ngày trùng lặp (nếu có, giữ lại giá trị cuối cùng)
    df = df.loc[~df.index.duplicated(keep='last')]
    
    # Sắp xếp Index theo thời gian
    df = df.sort_index()
    return df

# Áp dụng hàm làm sạch
df_oil_clean = prepare_daily_data(df_oil)
df_usdvnd_clean = prepare_daily_data(df_usdvnd)
df_vnindex_clean = prepare_daily_data(df_vnindex)

# 3. Hợp nhất Dữ liệu Hàng ngày (Outer Join)
# Outer Join đảm bảo giữ lại tất cả các ngày từ VNINDEX và các điểm thưa thớt từ OIL/USDVND
df_daily_merged = pd.merge(
    df_oil_clean,
    df_usdvnd_clean,
    left_index=True,
    right_index=True,
    how='outer'
)
df_daily_merged = pd.merge(
    df_daily_merged,
    df_vnindex_clean,
    left_index=True,
    right_index=True,
    how='outer'
)

# Định nghĩa quy tắc tổng hợp và resample từ từng ngày sang quý

In [110]:

# 4. Tổng hợp sang Tần suất Quý ('Q')
# Định nghĩa Quy tắc Tổng hợp (Aggregation Rules) dựa trên phân tích kinh tế
agg_rules = {
    # Mean: Cho giá hàng hóa, giúp làm phẳng biến động
    'oil_price_close': 'mean',      
    # Last: Cho Tỷ giá/Chỉ số, đại diện cho giá trị cuối kỳ
    'usd_vnd_close': 'last',        
    'vnindex_close': 'last'         
}

# Tổng hợp (Resample) sang tần suất Quý (kết thúc Quý) và áp dụng quy tắc
df_quarterly = df_daily_merged.resample('QS').agg(agg_rules)


df_quarterly_filtered = df_quarterly[
    (df_quarterly.index.year >= 2010) & 
    (df_quarterly.index.year <= 2024)
]


# Lưu file dữ liệu quý của 
- oil : Giá dầu thô
- usdnvd : Tỷ giá hối đoái (USD/VND)
- vnindex : Hiệu suất thị trường chứng khoán

In [111]:

# 5. Lưu và In Kết quả
df_quarterly_filtered.to_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\merged_daily_to_quarterly_final.csv')

# Đọc 2 file khi đã cùng tần suất

In [112]:
df_daily_to_quarterly = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\merged_daily_to_quarterly_final.csv' )
df_annual_to_quarterly = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\disaggregated_quarterly_data_fixed.csv' )

# Merge dữ liệu lại thành 1 

In [113]:
df_merged = pd.merge(
    df_daily_to_quarterly,
    df_annual_to_quarterly,
    left_on='date',
    right_on='date',
    how='inner'  # Chỉ giữ lại các quý có trong cả hai bộ dữ liệu
)

In [114]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          57 non-null     object 
 1   oil_price_close               37 non-null     float64
 2   usd_vnd_close                 37 non-null     float64
 3   vnindex_close                 57 non-null     float64
 4   fdi_inflow_quarterly          57 non-null     float64
 5   cpi_index_quarterly           57 non-null     float64
 6   gdp_growth_quarterly          57 non-null     float64
 7   retail_value_quarterly        49 non-null     float64
 8   unemployment_value_quarterly  57 non-null     float64
dtypes: float64(8), object(1)
memory usage: 4.1+ KB


In [115]:
df_merged.isnull().sum()

date                             0
oil_price_close                 20
usd_vnd_close                   20
vnindex_close                    0
fdi_inflow_quarterly             0
cpi_index_quarterly              0
gdp_growth_quarterly             0
retail_value_quarterly           8
unemployment_value_quarterly     0
dtype: int64

- Nhận thấy dữ liệu của usd_vnd và oil_price thiếu 

In [116]:
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged.set_index('date', inplace=True)

# Xử lý dữ liệu nan :
- Chỉ lấy dữ liệu từ 2015 đến 2024

In [117]:
df_merged = df_merged[
    (df_merged.index.year >= 2015) & 
    (df_merged.index.year <= 2024)
]


# Dữ liệu sẽ gồm 37 quý từ đầu năm 2015 đến đầu năm 2024


In [118]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37 entries, 2015-01-01 to 2024-01-01
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   oil_price_close               37 non-null     float64
 1   usd_vnd_close                 37 non-null     float64
 2   vnindex_close                 37 non-null     float64
 3   fdi_inflow_quarterly          37 non-null     float64
 4   cpi_index_quarterly           37 non-null     float64
 5   gdp_growth_quarterly          37 non-null     float64
 6   retail_value_quarterly        37 non-null     float64
 7   unemployment_value_quarterly  37 non-null     float64
dtypes: float64(8)
memory usage: 2.6 KB


In [119]:

df_merged = df_merged.reset_index()

features = pd.DataFrame(index=df_merged.index)

# Feature 1: Đà tăng trưởng quý trước
df_merged['gdp_growth_lag_1'] = df_merged['gdp_growth_quarterly'].shift(1)

# Feature 2: Yếu tố mùa vụ (cùng kỳ năm trước)
df_merged['gdp_growth_lag_4'] = df_merged['gdp_growth_quarterly'].shift(4)

# Feature 3: Động lực đầu tư (FDI)
df_merged['fdi_inflow_lag_1'] = df_merged['fdi_inflow_quarterly'].shift(1)

# Feature 4: Động lực tiêu dùng (Bán lẻ)
df_merged['retail_value_qoq_change_lag_1'] = df_merged['retail_value_quarterly'].pct_change(1).shift(1)

# Feature 5: Biến tương tác (Giá dầu * Tỷ giá)
oil_price_in_vnd = df_merged['oil_price_close'] * df_merged['usd_vnd_close']
df_merged['oil_price_in_vnd_lag_1'] = oil_price_in_vnd.shift(1)


# 3.2. Loại bỏ các hàng có NaN (rất quan trọng)
# 4 quý đầu tiên sẽ bị NaN do phép .shift(4)
df_merged = df_merged.dropna()



In [120]:
df_merged.head()

,date,oil_price_close,usd_vnd_close,vnindex_close,fdi_inflow_quarterly,cpi_index_quarterly,gdp_growth_quarterly,retail_value_quarterly,unemployment_value_quarterly,gdp_growth_lag_1,gdp_growth_lag_4,fdi_inflow_lag_1,retail_value_qoq_change_lag_1,oil_price_in_vnd_lag_1
4,2016-01-01,35.243571,21950.0,561.22,3.015385e+09,2.348745,6.596191,105.93750,1.838250,6.875725,7.098584,3.023140e+09,-0.005479,9.903037e+05
5,2016-04-01,47.062969,21911.0,632.26,3.105128e+09,2.561747,6.658736,106.91250,1.844750,6.596191,7.024297,3.015385e+09,-0.027204,7.735964e+05
6,2016-07-01,46.950781,21797.0,685.73,3.194872e+09,2.774749,6.721282,107.88750,1.851250,6.658736,6.950011,3.105128e+09,0.009204,1.031197e+06
7,2016-10-01,51.072951,22468.0,664.87,3.284615e+09,2.987751,6.783827,108.86250,1.857750,6.721282,6.875725,3.194872e+09,0.009120,1.023386e+06
8,2017-01-01,54.496452,22435.0,722.31,3.398462e+09,3.512993,6.743384,111.75375,2.141375,6.783827,6.596191,3.284615e+09,0.009037,1.147507e+06


In [122]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 4 to 36
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           33 non-null     datetime64[ns]
 1   oil_price_close                33 non-null     float64       
 2   usd_vnd_close                  33 non-null     float64       
 3   vnindex_close                  33 non-null     float64       
 4   fdi_inflow_quarterly           33 non-null     float64       
 5   cpi_index_quarterly            33 non-null     float64       
 6   gdp_growth_quarterly           33 non-null     float64       
 7   retail_value_quarterly         33 non-null     float64       
 8   unemployment_value_quarterly   33 non-null     float64       
 9   gdp_growth_lag_1               33 non-null     float64       
 10  gdp_growth_lag_4               33 non-null     float64       
 11  fdi_inflow_lag_1          

# Lưu df

In [123]:
df_merged.to_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\final_clean_dataset.csv')